In [ ]:
import requests
import json
from time import sleep


# TEST CONVEYOR SCRIPT

## Pre-requisites

ShopCX/Vizix
-  the fixture MUST have the correct attribut (pack or ship)

HUB connected & configured

RFID reader connected to the HUB & configured

ASN id
- the refLis MUST have the same bizStep than the fixture

Tags 


## Principle

Request the HUB to compare the tags in the fields of the antennas & and the expected items in the reference list (ASN)

## Error Codes

0   Device is ready

100 Device Unknown 

101 Device KO 

102	Error, device already running

200	ASN Unknown

201	NOK: Expected and read items does not match

400	extended filter unknown

401	epcisresult filter unknown (only if device OK)

201	ASN already under processing

500	Unexpected error during the process

501	No EPCIS mapping for device

502	Failed to reach the hub

503	Connection with device closed early. Missing tags probable


In [ ]:
# HUP ID ADDRESS
hubIp = '10.30.11.1'

In [ ]:
# parameters for HUB API
port =  '8082'
headers = {'accept': 'application/json'}
headerWithJson = {'Content-Type':'application/json','accept': 'application/json'}
requestTimeOut = 10

In [ ]:
# check devices
status =  False
url = 'http://' + hubIp + ':'+port + '/supply-process/rest/devices'
print ("GET : "+url)
try :
    response = requests.get(url,headers=headers,verify=True, timeout=requestTimeOut)
    if response.status_code == 200:
        print('HTTP 200 RESPONSE')
        allDevices = response.json()
        print ("All devices")
        print(json.dumps(allDevices,indent=4))
        status = True

    elif response.status_code == 404:
        print('HTTP 404 RESPONSE')
        print(response.content.decode("utf-8"))
    else:
        print('HTTP '+str(response.status_code)+' RESPONSE')
        print(response.content.decode("utf-8"))
    
except requests.exceptions.RequestException as e:  
    print (e)
if status:
    print ("AUTOMATED PROCESS devices")
    for device in allDevices:
        if ( device.get("mode") == "AUTOMATED_PROCESS"):
            print(json.dumps(device,indent=4))

# Nominal Process example

In [ ]:
# Process properties
deviceId = '3663328.99999.2'
processTime = 50


In [ ]:
asnNumber = 'ElS1gMZCMBPizqNGgekdGrgu5w1hHmrQ' # replace by the correct ASN ID (number)

In [ ]:
# start process

status = False
payload ={
}
options = '?extended=reflist,epcis,compare&epcisresult=ok'

# reflist : HUB will send the refList definition
# epcis : 
# compare : 
# epcisresult = ok  :
# epcisresult = all :


url = 'http://' + hubIp + ':'+ port + '/supply-process/rest/devices/'+deviceId+'/start?asnId='+asnNumber
print ("POST : "+url)
response = requests.post(url, headers=headerWithJson,json=payload, verify=True, timeout=requestTimeOut)
if response.status_code == 200:
    print('HTTP 200, RESPONSE')
    jsonResponse = response.json()
    print(json.dumps(jsonResponse,indent=4))
    statusCode = jsonResponse.get('code')
    if (statusCode==0):
        print('OK :'+str(jsonResponse.get('message')))
        status = True
    else:
        print('KO because :'+str(jsonResponse.get('message')))
    
elif response.status_code == 404:
    print('HTTP 404 RESPONSE')
    print(response.content.decode("utf-8"))
else:
    print('HTTP '+str(response.status_code)+' RESPONSE')
    print(response.content.decode("utf-8"))

if (status):    
    # simulate the conveyor actions/timeout
    print("READING ...")

    sleep(processTime)
    print("...DONE")
    
    # stop process   
    
    url = 'http://' + hubIp + ':'+ port + '/supply-process/rest/devices/'+deviceId+'/stop'+options
    print ("POST : "+url)
    response = requests.post(url, headers=headers, verify=True, timeout=requestTimeOut)
    if response.status_code == 200:
        print('HTTP 200, RESPONSE')
        jsonResponse = response.json()
        statusCode = jsonResponse.get('code')
        if (statusCode==0):
            print('OK :'+str(jsonResponse.get('message')))
            print(json.dumps(response.json(),indent=4))
        elif (statusCode==201):
            print('NOK :'+str(jsonResponse.get('message')))
            print(json.dumps(response.json(),indent=4))
        else:
            print('KO because :'+str(jsonResponse.get('message')))
    
    elif response.status_code == 404:
        print(' HTTP 404, RESPONSE')
        print(response.content.decode("utf-8"))
    else:
        print(' HTTP '+str(response.status_code)+', RESPONSE')
        print(response.content.decode("utf-8"))

# Tests cases

## TEST with BAD asn id

In [ ]:
# Process properties
#deviceId = ''
asnNumber = 'NotExistingASN'

In [ ]:
# start process
status = False
payload ={
}
url = 'http://' + hubIp + ':'+ port + '/supply-process/rest/devices/'+deviceId+'/start?asnId='+asnNumber
print ("POST : "+url)
response = requests.post(url, headers=headerWithJson,json=payload, verify=True, timeout=requestTimeOut)
if response.status_code == 200:
    jsonResponse = response.json()
    print(' RESPONSE')
    print(json.dumps(response.json(),indent=4))
    statusCode = jsonResponse.get('code')
    if (!statusCode):
        print('KO Not Status Code !' )

    else if (statusCode==0):
        print('OK :'+jsonResponse.get('message'))
        status = True
    else:
        print('KO because :'+jsonResponse.get('message'))
    
elif response.status_code == 404:
    print(' RESPONSE')
    print(response.content.decode("utf-8"))
else:
    print(' RESPONSE')
    print(response.content.decode("utf-8"))


## TEST WithoutASN

In [ ]:
# Process properties
# deviceId = ''
processTime = 10 # in seconds

In [ ]:
# start process

status = False
payload ={
}
options = '?extended=epcis&epcisresult=ok'


url = 'http://' + hubIp + ':'+ port + '/supply-process/rest/devices/'+deviceId+'/start'
print ("POST : "+url)
response = requests.post(url, headers=headerWithJson,json=payload, verify=True, timeout=requestTimeOut)
if response.status_code == 200:
    print('HTTP 200, RESPONSE')
    jsonResponse = response.json()
    print(json.dumps(jsonResponse,indent=4))
    statusCode = jsonResponse.get('code')
    if (statusCode==0):
        print('OK :'+str(jsonResponse.get('message')))
        status = True
    else:
        print('KO because :'+str(jsonResponse.get('message')))
    
elif response.status_code == 404:
    print('HTTP 404 RESPONSE')
    print(response.content.decode("utf-8"))
else:
    print('HTTP '+str(response.status_code)+' RESPONSE')
    print(response.content.decode("utf-8"))

if (status):    
    # simulate the conveyor actions/timeout
    print("READING ....")

    sleep(processTime)
    
    # stop process   
    
    url = 'http://' + hubIp + ':'+ port + '/supply-process/rest/devices/'+deviceId+'/stop'+options
    print ("POST : "+url)
    response = requests.post(url, headers=headers, verify=True, timeout=requestTimeOut)
    if response.status_code == 200:
        print('HTTP 200, RESPONSE')
        jsonResponse = response.json()
        print(json.dumps(response.json(),indent=4))
        statusCode = jsonResponse.get('code')
        if (statusCode==0):
            print('OK :'+str(jsonResponse.get('message')))
        elif (statusCode==201):
            print('NOK :'+str(jsonResponse.get('message')))
        else:
            print('KO because :'+str(jsonResponse.get('message')))
    
    elif response.status_code == 404:
        print(' HTTP 404, RESPONSE')
        print(response.content.decode("utf-8"))
    else:
        print(' HTTP '+str(response.status_code)+', RESPONSE')
        print(response.content.decode("utf-8"))

## UTILS : Force Stop to clean

In [ ]:
# Process properties
if deviceId:
    # stop process   
    url = 'http://' + hubIp + ':'+ port + '/supply-process/rest/devices/'+deviceId+'/stop'
    print ("POST : "+url)
    response = requests.post(url, headers=headers, verify=True, timeout=requestTimeOut)
    if response.status_code == 200:
        print('HTTP 200, RESPONSE')
        jsonResponse = response.json()
        print(json.dumps(response.json(),indent=4))
        statusCode = jsonResponse.get('code')
        if (statusCode==0):
            print('OK :'+str(jsonResponse.get('message')))
            print(json.dumps(response.json(),indent=4))
        elif (statusCode==201):
            print('NOK :'+str(jsonResponse.get('message')))
            print(json.dumps(response.json(),indent=4))
        else:
            print('KO because :'+str(jsonResponse.get('message')))
    
    elif response.status_code == 404:
        print(' HTTP 404, RESPONSE')
        print(response.content.decode("utf-8"))
    else:
        print(' HTTP '+str(response.status_code)+', RESPONSE')
        print(response.content.decode("utf-8"))

## UTILS : create a RefList

In [ ]:
# Platform config
platformProtocol = 'https'
platformName = 'dpo-lab01.shopcx.io'
platformPort =  ''
platformToken = 'dGVzdGVyOnRlc3Rpc2xpZmU'
platformHeaders = {'Authorization':'Basic '+ PlatformToken,'accept': 'application/json'}
platformHeaderWithJson = {'Authorization':'Basic '+ PlatformToken, 'Content-Type':'application/json','accept': 'application/json'}
platformRequestTimeOut = 10

# premise (DC)
premiseBizLoc = 'CONDA'
bizStep = 'packing'

In [ ]:
import random
import string

# generate random asnId(must not exist)
asnNumber = ''.join([random.choice(string.ascii_letters + string.digits) for n in range(32)])
print("transactionId (asnNumber) : "+asnNumber)

In [ ]:
# RefList definition 
myRefList = {
        "contentFormat": "quantity",
        "transactionId": asnNumber,
        "status": "available",
        "bizStep": "urn:epcglobal:cbv:bizstep:"+bizStep,
        "bizLocation": premiseBizLoc,
        "containers": [
            {
                "content": [
                    {
                        "format": "quantity",
                        "gtin": "03663328000083",
                        "quantity": 1
                    },
                    {
                        "format": "quantity",
                        "gtin": "03663328000106",
                        "quantity": 1
                    },
                    {
                        "format": "quantity",
                        "gtin": "03663328000113",
                        "quantity": 1
                    },
                    {
                        "format": "quantity",
                        "gtin": "03663328012529",
                        "quantity": 2
                    }
                ]
            }
        ]
    }
print (json.dumps(myRefList,indent=4))

In [ ]:
# Send RefList to the Cloud
# PUT /supply-chain-api/rest/supply_chain/reference_lists
#json = myRefList
if asnNumber:
    url = platformProtocol+'://' + platformName  + '/supply-chain-api/rest/supply_chain/reference_lists'
    print ("PUT : "+url)
    #print (myRefList)
    response = requests.put(url, headers=platformHeaderWithJson,json = myRefList , verify=True, timeout=platformRequestTimeOut)
    referenceListId = response.text
    print("asnNumber: "+asnNumber+" referenceListId:" + referenceListId)

### check by referenceListId

In [ ]:
if referenceListId:
    url = platformProtocol+'://' + platformName  + '/supply-chain-api/rest/supply_chain/reference_lists/'+referenceListId
    print ("GET : "+url)
    response = requests.get(url, headers=platformHeaderWithJson , verify=True, timeout=platformRequestTimeOut)
    jsonResponse = response.json()
    print(json.dumps(jsonResponse,indent=4))

### check by asn Number (Transaction Id)

In [ ]:
if asnNumber:
    data = {
        "filters":[
            {
                "property": "transactionId",
                "operator":"EQ",
                "values":[asnId]
            }
        ]
    }
    
    url = platformProtocol+'://' + platformName  + '/supply-chain-api/rest/supply_chain/reference_lists/searches'
    print ("POST : "+url)
    response = requests.post(url, headers=platformHeaderWithJson ,json=data, verify=True, timeout=requestTimeOut)
    jsonResponse = response.json()
    print(json.dumps(jsonResponse,indent=4))
    
    

## UTILS : check Events in the platform

In [ ]:
if asnNumber :
    data = {
        "filters": [
            {
                "property": "bizStep",
                "operator": "EQ",
                "values": [
                    "urn:epcglobal:cbv:bizstep:"+bizStep
                ]
            },
            {
                "property": "bizTransList.value",
                "operator": "EQ",
                "values": [
                    asnId
                ]
            }
        ],
        "order": {
            "property": "eventTime",
                  "direction": "DESC"
        },
        "pagination": {
            "documents": 100,
            "page": 0
        }
    }
    url = platformProtocol+'://' + platformName  + '/epcis-core/rest/events/searches'
    print ("POST : "+url)
    print(json.dumps(data,indent=4))

    response = requests.post(url, headers=platformHeaderWithJson ,json=data, verify=True, timeout=requestTimeOut)
    jsonResponse = response.json()
    print ("RESULT : ")
    print(json.dumps(jsonResponse,indent=4))



In [ ]:
# DRAFT
if asnNumber :
    data = {
        "filters":[
            {
                "property": "transactionId",
                "operator":"EQ",
                "values":[asnNumber]
            }
        ]
    }
    
    url = platformProtocol+'://' + platformName  + '/epcis-core/rest/events/searches'
    print ("POST : "+url)
    response = requests.post(url, headers=platformHeaderWithJson ,json=data, verify=True, timeout=requestTimeOut)
    jsonResponse = response.json()
    print(json.dumps(jsonResponse,indent=4))
    